# Launch: AI YouTube Shorts Generator — Advanced (Colab)
Run this in Google Colab.


In [ ]:
# Setup
!apt-get update -qq
!apt-get install -y -qq ffmpeg imagemagick fonts-freefont-ttf > /dev/null
!sed -i 's/<policy domain="path" rights="none" pattern="@\*" />/<!-- <policy domain="path" rights="none" pattern="@\*" /> -->/g' /etc/ImageMagick-6/policy.xml || true
%pip -q install --upgrade pip wheel setuptools
%pip -q install gradio==4.* moviepy==1.0.3 imageio-ffmpeg
%pip -q install numpy<2.0 opencv-python-headless pytubefix pydub pysrt faster-whisper google-generativeai
%pip -q install 'openai>=1.35.0'
import torch, os
if torch.cuda.is_available():
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    %pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
else:
    %pip -q install torch torchvision torchaudio
!mkdir -p models
!wget -q -O models/haarcascade_frontalface_default.xml https://raw.githubusercontent.com/faisu223/Faisal/feature/gradio-interface/haarcascade_frontalface_default.xml
print('Setup complete. CUDA:', torch.cuda.is_available())


In [ ]:
# Write modules into Colab runtime (if not already present).
import pathlib, json
from pathlib import Path
mods={'subs_utils.py': '''REPLACE_SUBS_UTILS''', 'video_utils.py': '''REPLACE_VIDEO_UTILS''', 'llm_utils.py': '''REPLACE_LLM_UTILS''', 'pipeline_advanced.py': '''REPLACE_PIPELINE_ADV''' }
for name, content in mods.items():
    p=Path(name);
    if not p.exists():
        p.write_text(content)
print('Modules ready.')


In [ ]:
# Gradio UI launcher
import gradio as gr
from pipeline_advanced import generate_pipeline

def run_ui():
    with gr.Blocks() as demo:
        gr.Markdown('# AI YouTube Shorts Generator — Advanced')
        with gr.Row():
            with gr.Column(scale=1):
                youtube_url = gr.Textbox(label='YouTube URL (optional)')
                video_file = gr.File(label='Or upload a video')
                srt_file = gr.File(label='Upload SRT (optional)')
                provider = gr.Dropdown(['OpenAI','Gemini'], label='AI Provider', value='OpenAI')
                openai_key = gr.Textbox(label='OpenAI API Key', type='password', visible=True)
                gemini_key = gr.Textbox(label='Gemini API Key', type='password', visible=False)
                with gr.Row():
                    min_len = gr.Slider(5,120,value=25,step=1,label='Min length (s)')
                    max_len = gr.Slider(10,180,value=60,step=1,label='Max length (s)')
                max_clips = gr.Slider(1,10,value=3,step=1,label='Max clips')
                aspect = gr.Dropdown(['9:16','16:9','1:1'], value='9:16', label='Aspect ratio')
                crop_mode = gr.Dropdown(['Center','Face-track'], value='Face-track', label='Crop mode')
                karaoke = gr.Checkbox(label='Karaoke word-level subtitles', value=True)
                export_srt = gr.Checkbox(label='Export SRT + transcription in ZIP', value=True)
                title_mode = gr.Dropdown(['Auto','Custom','None'], value='Auto', label='Title overlay')
                custom_title = gr.Textbox(label='Custom title (if mode=Custom)')
                platform = gr.Dropdown(['TikTok','YouTube','Generic'], value='TikTok', label='Platform (safe margins)')
                out_prefix = gr.Textbox(label='Output name prefix', value='short')
                watermark_file = gr.File(label='Watermark image (optional)')
                go = gr.Button('Generate Shorts')
            with gr.Column(scale=1):
                logs = gr.Textbox(label='Logs', lines=18)
                out_zip = gr.File(label='Results ZIP')
        def _toggle(p): return (gr.update(visible=p=='OpenAI'), gr.update(visible=p=='Gemini'))
        provider.change(_toggle, inputs=provider, outputs=[openai_key, gemini_key])
        def _run(*args):
            zip_path = generate_pipeline(*args, logger=print)
            log_text = 'Done.' if zip_path else 'Failed to generate.'
            return zip_path, log_text
        go.click(_run, [youtube_url, video_file, srt_file, provider, openai_key, gemini_key, min_len, max_len, max_clips, aspect, crop_mode, karaoke, export_srt, title_mode, custom_title, platform, out_prefix, watermark_file], [out_zip, logs])
    return demo

demo = run_ui()
demo.launch(debug=True, share=True)
